In [2]:
using Distributions
σ(x) = 1.0/(1.0+exp(-x));
ϕ(x) = -log(1.0/x -1.0);

In [ ]:
# unknowns
n = 3;

# equations
m = 2;

# valid values of the unknowns
xs = rand(n);

# weights of a neural net layer
W  = rand(Uniform(-sqrt(n+1),sqrt(n+1)), (n+1,m));

# real estimation of xs flowing through the layer
ys = map(σ, W'*[1.0;xs]);

In [ ]:
function narrowBounds(boundsA, boundsB)
    lowBounds  = Array{Float64,1}(length(boundsA)+1)
    highBounds = Array{Float64,1}(length(boundsB)+1)
    lowBounds[1]  = 0.0
    highBounds[1] = 1.0
    for i in eachindex(boundsA)[1:end]
        if boundsA[i] < boundsB[i]
            lowBounds[i+1]  = boundsA[i]
            highBounds[i+1] = boundsB[i]
        else
            lowBounds[i+1]  = boundsB[i]
            highBounds[i+1] = boundsA[i]
        end
    end
    (maximum(lowBounds),minimum(highBounds))
end

In [ ]:
function nextInterval(ys, W, B)
    variables, equations = size(W)
end

In [ ]:
function sendOutput(ys, W)
    # number of variables and equations
    variables, equations = size(W)
    
    # xs are the values to be calculated (except x[1] which is 1.0)
    xs = ones(variables)
    # ks are the constant values in the equations
    ks = (map(ϕ,ys).-W[1,:]')[:]
    
    negativeWeight = zeros(ks)
    positiveWeight = zeros(ks)
    for j in 1:equations
        for i in 2:variables
            w = W[i,j]
            w < 0 ? negativeWeight[j] += w : positiveWeight[j] += w
        end
    end
    
    knownValues = zeros(equations)
    for i in 2:variables
        reciprocals = zeros(equations)
        for j in 1:equations
            W[i,j] < 0 ? negativeWeight[j] -= W[i,j] : positiveWeight[j] -= W[i,j]
            reciprocals[j] = 1.0/W[i,j]
        end
        boundsA = (ks .- negativeWeight .- knownValues).*reciprocals
        boundsB = (ks .- positiveWeight .- knownValues).*reciprocals
        lowBound, highBound = narrowBounds(boundsA, boundsB)
        println("xs[$i] ∈ [$lowBound,$highBound]")
        xs[i] = rand(Uniform(lowBound, highBound))
        for j in 1:equations
            knownValues[j] += xs[i]*W[i,j]
        end
    end
    xs
end

In [ ]:
sendOutput(ys, W)

In [ ]:
function sendOutput(ys, W)
    equations = size(W)[2]
    variables  = size(W)[1]
    
    xs = zeros(variables)           # Array{Float64,1}
    ks = (map(ϕ, ys) - W[1,:]')[:]  # Array{Float64,1}
    
    xs[1] = 1.0                     # first entry is always 1.0
    
    negativeWeight = zeros(ks)
    positiveWeight = zeros(ks)
    for i in 1:size(W)[1]
        for j in 1:size(W)[2]
            w = W[i,j]
            w < 0 ? negativeWeight[j] += w : positiveWeight[j] += w
        end
    end
    
    knownValues = zeros(ks)
    for row in 2:variables
        weights = W[row,:]
        for col in eachindex(weights)
            w = weights[col]
            w < 0 ? negativeWeight[col] -= w : positiveWeight[col] -= w
        end
        reciprocals = (1.0./weights')[:]
        boundsA = (ks .- negativeWeight .- knownValues).*reciprocals
        boundsB = (ks .- positiveWeight .- knownValues).*reciprocals
        lowBound, highBound = narrowBounds(boundsA, boundsB)
        println(lowBound)
        println(highBound)
        println()
        xs[row] = rand(Uniform(lowBound, highBound))
        knownValues = knownValues .+ (xs[row] .* weights)
    end
    xs
end

In [ ]:
y = [20,-15]
A = [2 -3 5; -1 2 -9]

In [ ]:
pinv(A)*y

In [ ]:
eye(size(A)[2]) - (pinv(A)*A)

In [ ]:
x = pinv(A)*y + (eye(size(A)[2]) - pinv(A)*A)*[0,0,0]

In [ ]:
A*x

In [ ]:
([0,0,0]-pinv(A)*y)

In [ ]:
(eye(size(A)[2]) - pinv(A)*A)

In [ ]:
([0,0,0]-pinv(A)*y)\(eye(size(A)[2]) - pinv(A)*A)

In [ ]:
[-2.70309e-16,  2.47783e-16,  -2.47783e-16]

In [ ]:
x = pinv(A)*y + (eye(size(A)[2]) - pinv(A)*A)*[-2.70309e-16,2.47783e-16,-2.47783e-16]

In [ ]:
A*x

In [ ]:
([1,1,1]-pinv(A)*y)\(eye(size(A)[2]) - pinv(A)*A)

In [ ]:
x = pinv(A)*y + (eye(size(A)[2]) - pinv(A)*A)*[0.0406416,0.0310789,0.00239068]

In [ ]:
A*x

In [ ]:
y\A

In [ ]:
###
### Planteamiento del problema como red neuronal
###

# unknowns
n = 2;

# equations
m = 1;

# valid values of the unknowns
xs = rand(n);

# weights of a neural net layer
W  = rand(Uniform(-sqrt(n+1),sqrt(n+1)), (n+1,m));

# real estimation of xs flowing through the layer
ys = map(σ, W'*[1.0;xs]);
;
# Conociendo W y ys, encontrar xs que cumplan ys = map(σ, W'*[1.0;xs])

In [ ]:
###
### Planteamiento del problema como álgebra lineal
###

A = W'[:,2:end];
b = ys-W'[:,1]
;
# Encontrar z tal que Az = b

In [ ]:
A

In [ ]:
b

In [ ]:
pinv(A)

In [ ]:
x = pinv(A)*b + (eye(size(A)[2]) - pinv(A)*A)*[0,0]

In [ ]:
abs(A*x-b.-0)

In [ ]:
eye(size(A)[2]) - pinv(A)*A

In [ ]:
-pinv(A)*b

In [ ]:
let B = eye(size(A)[2]) - pinv(A)*A,
    x2 = [0,0],
    x3 = [1,1],
    inv = pinv(A)*b,
    v2 = x2 - inv,
    v3 = x3 - inv
    
    w2 = v2\B
    w3 = v3\B
    
    newX2 = inv + w2*B
    newX3 = inv + w3*B
end

In [ ]:
function rref{T}(A::Matrix{T})
    nr, nc = size(A)
    U = copy!(similar(A, T <: Complex ? Complex128 : Float64), A)
    e = eps(norm(U,Inf))
    i = j = 1
    while i <= nr && j <= nc
        (m, mi) = findmax(abs(U[i:nr,j]))
        mi = mi+i - 1
        if m <= e
            U[i:nr,j] = 0
            j += 1
        else
            for k=j:nc
                U[i, k], U[mi, k] = U[mi, k], U[i, k]
            end
            d = U[i,j]
            for k = j:nc
                U[i,k] /= d
            end
            for k = 1:nr
                if k != i
                    d = U[k,j]
                    for l = j:nc
                        U[k,l] -= d*U[i,l]
                    end
                end
            end
            i += 1
            j += 1
        end
    end
    U
end

In [ ]:
rref([A b])

In [ ]:
let FGH = svd(A),
    F = FGH[1],
    G = FGH[2],
    H = FGH[3]
    
    println(F)
    println()
    println(G)
    println()
    println(H)
end

In [23]:
function solutions(equations, unknowns)
    # Neural network
    weights     = rand(Uniform(-sqrt(unknowns+1),sqrt(unknowns+1)), (unknowns+1,equations))
    activations = rand(unknowns)
    results     = map(σ, weights'*[1.0;activations])
    # Linear algebra
    matrix      = weights'[:,2:end]
    solution    = results - weights'[:,1]
    
    inverse = pinv(matrix)
    
    A = inverse*solution
    B = I-inverse*matrix
    
    
    
    function f(w)
        I = eye(unknowns)
        #println("$(size(inverse)) * $(size(solution)) + ($(size(I)) - $(size(inverse)) * $(size(matrix))) * $(size(w))")
        A + B*w
    end
    
    return f, weights, results
end

solutions (generic function with 1 method)

In [24]:
f, W, ys = solutions(2, 4)

(f,
5x2 Array{Float64,2}:
  2.05965   -0.966901
 -0.876383   2.07872 
 -0.86542    1.24307 
  0.445383   2.11353 
 -1.94772   -0.356247,

[0.7322967432942546,0.7007621567768642])

In [25]:
f([0, 0, 0, 0])

4-element Array{Float64,1}:
 0.46291 
 0.348579
 0.189708
 0.361701

In [26]:
f([1, 0, 0, 0])

4-element Array{Float64,1}:
  0.976768 
  0.0180058
 -0.148511 
  0.20003  

In [27]:
f([-1, 0, 0, 0])

4-element Array{Float64,1}:
 -0.0509491
  0.679151 
  0.527927 
  0.523373 

In [28]:
f([0, 1, 0, 0])

4-element Array{Float64,1}:
 0.132337 
 1.10531  
 0.0320879
 0.138167 

In [29]:
W

5x2 Array{Float64,2}:
  2.05965   -0.966901
 -0.876383   2.07872 
 -0.86542    1.24307 
  0.445383   2.11353 
 -1.94772   -0.356247

In [30]:
ys

2-element Array{Float64,1}:
 0.732297
 0.700762

In [33]:
map(σ, W'*[1.0, 0.46291, 0.348579, 0.189708, 0.361701])

2-element Array{Float64,1}:
 0.675309
 0.668357